### Load Data

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import json
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from sentence_transformers import SentenceTransformer
import cohere

# Load the .env file
load_dotenv()


/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  'min_anystr_length': 'str_min_length',


True

### Instantiate Vector DB and LLM

In [3]:
api_key = os.getenv('QDRANT_API_KEY')


client = QdrantClient(
    url="https://8999b86c-f8b2-4d60-bdfa-8c68d39daae7.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key=api_key,timeout=200
)

print(api_key)

r09oP5L3SaVcMi_MvBC3yBOuRpcYChNeZjzTiEHr5cyoyHiP00CyEw


In [4]:
#Instantiate LLM

cohere_api_key = os.getenv('COHERE_API_KEY')
co = cohere.Client(cohere_api_key)


In [5]:
#Load Data
with open('diabetes_data_with_vectors', 'r') as file:
    diabetes_data_with_vectors = json.load(file)

### Develop Prompt

In [8]:
prompt_template = '''
You are emulating a diabetes patient.
Formulate just 4 insightful questions closely related to this with a very high cosine similarity diabetes conversation record. 
Use the context provided to generate relevant questions, but avoid directly copying words from the response.
Dont't repeat question
Record:
Context: {question}
return result in this format
[question1, question2, question3]

Dont include the instruction or anyother thing in the response
Return only the generated response as a python list


'''

In [9]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    result = co.chat(message= prompt).text

    json_response = result
    return json_response

In [12]:
import random
#Select 40 Random Data to evaluate Retrieval
sampledata = random.sample(diabetes_data_with_vectors, 40)


In [13]:
#Generate The List

results = {}
for doc in tqdm(sampledata): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions
    if doc_id == 'bedc96fa':
        print(questions)

  0%|          | 0/40 [00:00<?, ?it/s]

### Create Dataframe for retrieval evaluation

In [150]:
results[ids[33]] = "['How can I manage my diabetes effectively on a daily basis?', 'What are the potential risks if I dont inform others about my diabetes?', 'Are there any legal implications if I dont disclose my diabetes to, say, my employer or my school?', 'In what specific emergency situations would sharing my diabetes status be beneficial?', 'How can I explain my diabetes to those close to me in a simple and understandable way?']"

In [14]:
import ast

In [15]:
ids= list(results.keys())
main_rows = []

for document in tqdm(ids):
    for question in ast.literal_eval(results[document]):
        main_rows.append((question,document))
        
        
df = pd.DataFrame(main_rows, columns=["question", "id"])


  0%|          | 0/40 [00:00<?, ?it/s]

In [16]:
df.to_csv('retrieval_evaluation.csv', index=False)

In [18]:
len(df)

160

### Evaluate Retrieval

In [19]:
def search_query(query_vector):
    hits = client.search(
        collection_name="diabetes",
        query_vector=query_vector,
        limit=5
    )
    return hits

In [20]:
data = pd.read_csv('retrieval_evaluation.csv')

In [21]:
retrieval_evaluation_dict = data.to_dict(orient = 'records')


In [22]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [23]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


In [24]:
def retrieval_evaluator(data_dictionary):
    relevance_total = []

    for question in tqdm(data_dictionary):
        question_id = question['id']
        vector = embedding_model.encode(question['question'])
        results = search_query(vector)
        relevance = [d.payload['id'] == question_id for d in results]
        relevance_total.append(relevance)
        
    return hit_rate(relevance_total),mrr(relevance_total)

In [25]:
hitrate,mrr = retrieval_evaluator(retrieval_evaluation_dict)
    

  0%|          | 0/160 [00:00<?, ?it/s]

In [26]:
print(f'Hit Rate Is : {hitrate}')
print(f'MRR Is : {mrr}')

Hit Rate Is : 0.6375
MRR Is : 0.4592708333333335


In [209]:
Hit Rate Is : 0.47398843930635837
MRR Is : 0.2645472061657033

0.4508670520231214
0.2784200385356455
